In [9]:
import json
import os

all_data = []

for file in os.listdir('Raw-data'):
    if not file.endswith('json'):
        continue
    with open('Raw-data/' + file) as f:
        all_data.extend(json.load(f))


all_data_ids = [x['id'] for x in all_data]
# check unique ids
assert len(all_data_ids) == len(set(all_data_ids))

In [10]:
with open('HumorCorpusFinal.txt', 'r') as f:
    humor_corpus = [line.split() for line in f.readlines()]
humor_corpus = [l for l in humor_corpus if len(l) == 2]

ids = [l[0] for l in humor_corpus]
# check no duplicates
# print(ids)
# assert len(ids) == len(set(ids)), (len(ids), len(set(ids)))


# print(len(humor_corpus))
found_count = 0
not_found = 0
for id, label in humor_corpus:
    found = False
    for data in all_data:

        if data['id'] == id:
            if 'label' in data:
                print('DUPLICATE')
                assert data['label'] == label
            data['label'] = label
    
            # print('FOUND')
            found = True

            break
    if found:
        found_count += 1
    else:
        not_found += 1
        print(id)
    
    



In [12]:
positive_examples = [d for d in all_data if 'label' in d and d['label'].lower() == 'h']
negative_examples = [d for d in all_data if 'label' in d and d['label'].lower() == 'n']

print(len(positive_examples), len(negative_examples), found_count)



1759 1192 2952


In [15]:
# divide data into train, dev, test  with 60, 20, 20 split
import random
random.seed(42)

random.shuffle(positive_examples)
random.shuffle(negative_examples)

train_pos = positive_examples[:int(len(positive_examples) * 0.6)]
dev_pos = positive_examples[int(len(positive_examples) * 0.6):int(len(positive_examples) * 0.8)]
test_pos = positive_examples[int(len(positive_examples) * 0.8):]

train_neg = negative_examples[:int(len(negative_examples) * 0.6)]
dev_neg = negative_examples[int(len(negative_examples) * 0.6):int(len(negative_examples) * 0.8)]
test_neg = negative_examples[int(len(negative_examples) * 0.8):]

train = train_pos + train_neg
dev = dev_pos + dev_neg
test = test_pos + test_neg

random.shuffle(train)
random.shuffle(dev)
random.shuffle(test)

with open('train_new.tsv', 'w') as f:
    for d in train:
        # map h to 1 and n to 0
        label = 1 if d['label'].lower() == 'h' else 0
        f.write(f"{d['id']}\t{' '.join(d['text'].split())}\t{label}\n")

with open('dev_new.tsv', 'w') as f:
    for d in dev:
        label = 1 if d['label'].lower() == 'h' else 0
        f.write(f"{d['id']}\t{' '.join(d['text'].split())}\t{label}\n")


with open('test_new.tsv', 'w') as f:
    for d in test:
        label = 1 if d['label'].lower() == 'h' else 0
        f.write(f"{d['id']}\t{' '.join(d['text'].split())}\t{label}\n")

